# Lights, Camera, ReAction 🎬🎥😆


**ADD HOW INFO IS STORED IN META DATA**

This notebook builds a full **sitcom script generation pipeline** powered by **ReAct-based agents** — a powerful framework that combines **reasoning** (thinking through a problem) with **acting** (taking structured steps).

We start by **generating a sitcom concept** from creative keywords, then **outlining** the pilot episode scene-by-scene.  
**Scene 1** is generated directly from the outline to establish the world and tone.  
After that, each new scene is **scripted, reviewed, and improved** using specialized **ReAct agents** — a **Character Agent**, **Comedy Agent**, and **Environment Agent** — that simulate a real sitcom writers' room.

---

 **Benefits of using ReAct agents**:

- **More structured and transparent thinking:** Agents reason step-by-step before making edits.
- **Dynamic adaptation:** Agents flexibly plan the next creative moves based on scene history.
- **Better long-term coherence:** Scenes evolve logically, with tracked character growth, running jokes, emotional arcs, and worldbuilding.

---

After generation, each scene is **summarized and stored in a vector database**, enabling fast retrieval of scene metadata for future story planning.  
By combining **structured agent workflows** and **retrieval-augmented memory**, we bring sitcom worlds to life — one coherent, character-driven scene at a time.


In [1]:
!pip install datasets
!pip install sentence-transformers faiss-cpu

### Mounting Drive and Appending System

In [1]:
from google.colab import drive
import sys

drive.mount('/content/drive')
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils")
sys.path.append("/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils/agents")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# TO DELETE

!ls "/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils"


agents		       __pycache__	  script_review.py  vector_db_utils.py
outline_generation.py  screen_writing.py  text_utils.py


In [3]:
import importlib
import agent_evaluation

importlib.reload(agent_evaluation)


<module 'agent_evaluation' from '/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils/agents/agent_evaluation.py'>

In [4]:
from agents.character_agent import CharacterAgent
from agents.comedy_agent import ComedicAgent
from agents.environment_agent import EnvironmentAgent
from agents.scene_planner_agent import ScenePlannerAgent

from agent_evaluation import (
    evaluate_character_agent_scene,
    evaluate_comedic_agent_scene,
    evaluate_environment_agent_scene
)

from text_utils import (
    display_markdown_output,
    extract_scene,
    extract_title,
)

from outline_generation import (
    generate_sitcom_pitch,
    generate_pilot_episode_outline
)

from script_review import (
    validate_episode_outline,
)

from screen_writing import (
    generate_scene_1_script,
    generate_scene
)

from vector_db_utils import (
    summarize_scene,
    add_scene_to_vector_db
)

In [5]:
import numpy as np
import openai
from openai import OpenAI
from IPython.display import Markdown, display
import re
import time
from datetime import timedelta
from google.colab import userdata
import os
from sentence_transformers import SentenceTransformer
import faiss

In [6]:
api_key = userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)

### Vector Database

This block initializes the sentence embedding model and sets up a FAISS index to store vector representations of scenes. The `all-MiniLM-L6-v2` model encodes scenes into fixed-size embeddings, and the FAISS index enables fast vector operations. The `scene_metadata` list stores contextual information linked to each scene embedding.


In [54]:
# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Vector store initialization
dimension = embedding_model.get_sentence_embedding_dimension()
index = faiss.IndexFlatL2(dimension)

# To track metadata
vector_metadata = []


### 🎬 Generating the Sitcom Pitch

This step generates a sitcom concept using the `generate_sitcom_pitch` function. The function takes in a dictionary of categorized keywords (e.g., setting, characters, themes, tone/genre) to guide the generation process.

- The language model plays the role of a **professional screenwriter**.
- It produces a 1-paragraph pitch that includes a **title** and a **detailed premise**, highlighting the setting, main characters, their dynamics, and the overall tone of the show.
- Keywords help ground the pitch in user-defined ideas, ensuring thematic and narrative consistency.

In [8]:
keywords = {
    "setting": ["urban locksmith shop", "New York City"],
    "characters": ["ex-con protagonist", "estranged daughter", "quirky parole officer"],
    "themes": ["second chances", "odd couple dynamic"],
    "tone_genre": ["buddy comedy", "heartfelt absurdism"]
}


**TODO**

Make sure the title is quotation marks

In [9]:
# Generating Sitcom Pitch
sitcom_pitch = generate_sitcom_pitch(client, keywords)
display(Markdown(f"### Sitcom Concept:\n\n{sitcom_pitch}"))


### Sitcom Concept:

Title: "Second Lock"

"Second Lock" is an absurd yet heartwarming buddy comedy set in an urban locksmith shop in New York City. The show follows our ex-con protagonist, Jimmy "The Key" Kellerman, who is trying to genuinely reform his life and run his newly inherited locksmith shop. This is made complicated by his estranged, whip-smart daughter, Zoe, who unexpectedly moves in with him and isn’t afraid to challenge her dad's old ways of thinking. Adding to the madness is Steve, Jimmy's eccentric parole officer, who takes the "buddy" part of his job too seriously, frequently dropping by the shop to meddle – and often causing more trouble than he prevents. With an underlying theme of second chances and a heavy odd-couple dynamic, "Second Lock" combines heartfelt moments with hilarious hijinks, as these three misfits navigate their way through locked doors, both literal and metaphorical, on their journey towards a second chance at life.

### 📝 Generating a Pilot Episode Outline

The next part is to generate an episode outline for the pilot of the show, which is done using the `generate_pilot_episode_outline` function. This function takes in the sitcom pitch and produces a structured scene-by-scene breakdown of the pilot episode, simulating the voice of a **professional sitcom writer** preparing a network pitch.

- Begins with a concise **Episode Concept** (1–2 sentences).
- Breaks the episode into approximately 20 **numbered scenes**, each with a **title in quotation marks** and a brief description.
- Each scene summary includes what happens, who is involved, and the intended tone (e.g., funny, awkward, heartfelt).

This outline provides the narrative foundation for generating full scene scripts in the next stage.


In [10]:
outline = generate_pilot_episode_outline(client, sitcom_pitch)
display(Markdown(f"### Sitcom Outline:\n\n{outline}"))

### Sitcom Outline:

Episode Concept: In the pilot episode, Jimmy struggles to balance the demands of running his locksmith shop, dealing with the unexpected arrival of his estranged daughter, Zoe, and managing the overbearing presence of his parole officer, Steve.

Scene 1: "Opening Lock" Jimmy is seen at his locksmith shop, expertly crafting a lock, demonstrating his masterful skills and setting the tone for the unique setting.

Scene 2: "Unexpected Key" Zoe unexpectedly shows up at Jimmy's shop, surprising him and setting up their awkward reunion.

Scene 3: "The First Challenge" Zoe challenges Jimmy's traditional thinking by suggesting improvements to his locksmith shop, creating a tension-filled funny moment. 

Scene 4: "Buddy Parole Officer" Steve, Jimmy's parole officer, shows up at the shop, showing his oddball personality and overbearing nature.

Scene 5: "Living Quarters" Jimmy shows Zoe her old bedroom at his modest apartment, sparking a heartfelt conversation about their past. 

Scene 6: "Lock of the Past" Jimmy finds an old lock he made for Zoe, leading to a nostalgic and emotional moment. 

Scene 7: "First Customer Interaction" Jimmy deals with a difficult customer at the shop, showcasing his patience and flair for comedy.

Scene 8: "Parole Meeting" Steve conducts an official parole check at the shop, leading to awkward and funny misunderstandings. 

Scene 9: "Zoe's First Day" Zoe starts working at the shop and struggles with her first customer, creating a hilarious situation.

Scene 10: "Customer Complaint" A disgruntled customer returns, causing Jimmy and Zoe to work together, improving their relationship.

Scene 11: "A Serious Conversation" Jimmy and Zoe have a heartfelt conversation about why she returned, adding depth to their relationship.

Scene 12: "Steve's Intervention" Steve drops by unexpectedly, causing chaos at the shop with his well-meaning but misguided help.

Scene 13: "Locksmith Lessons" Jimmy teaches Zoe how to make a lock, leading to funny mishaps and a bonding moment. 

Scene 14: "Customer Triumph" Zoe successfully helps a customer, showing growth and earning her dad's approval. 

Scene 15: "Shop Closing" Jimmy and Zoe close the shop for the day, ending with a heartwarming moment of newfound understanding and respect.

Scene 16: "Steve's Apology" Steve apologizes for his earlier interference, resulting in a funny scene of him trying to make amends.

Scene 17: "Father-Daughter Dinner" Jimmy and Zoe share a quiet dinner, leading to more heartfelt conversation and bonding. 

Scene 18: "Late Night Customer" A late-night customer leads to another comedic event where Jimmy shows his expertise.

Scene 19: "Zoe's Revelation" Zoe confesses that she wants to stay and learn the locksmith trade, leading to a surprise for Jimmy and a touching moment. 

Scene 20: "Closing Keys" The day ends with Jimmy, Zoe and Steve sharing a goofy, light-hearted moment at the shop, setting the stage for future episodes.

### ✅ Episode Validation

To ensure that the sitcom pilot outline is coherent and aligns with the original pitch, we use the `validate_episode_outline` function. This function simulates the voice of a **veteran sitcom writer** who evaluates whether the episode outline makes narrative sense and fits the tone and premise.

- Checks for overall **coherence**, including consistency of tone, logical character progression, and structural viability.
- Returns a clear **"Yes" or "No"** on coherence, along with 2–4 bullet points of reasoning.
- If the episode is not coherent, it also provides concrete suggestions for improvement.

This validation step helps maintain narrative quality before moving into full scene scripting.


**TODO**

- Make sure it can re-generate the script
- Delte timedelta

In [11]:
validation = validate_episode_outline(client, sitcom_pitch, outline)
display(Markdown(f"### Coherence Evaluation:\n\n{validation}"))

### Coherence Evaluation:

Coherence: Yes

Reasoning:
- The outline aligns well with the sitcom pitch, including the main characters, setting, and theme of second chances. 
- The progression of the plot is logical and consistent. The episode begins with the surprise arrival of Zoe and ends with her decision to stay and learn the trade, which is a clear and satisfying arc for a pilot episode.
- The balance between comedic and heartfelt moments is well maintained, reflecting the intended tone of the sitcom.
- The characters are introduced and developed appropriately. Jimmy's skills and patience, Zoe's assertiveness and adaptability, and Steve's eccentricity and overbearing nature are all clearly demonstrated.

## Scene 1 Generation

In [12]:
scene_1_desc = extract_scene(outline, 1)
display(Markdown(scene_1_desc))

Scene 1: "Opening Lock" Jimmy is seen at his locksmith shop, expertly crafting a lock, demonstrating his masterful skills and setting the tone for the unique setting.

In [13]:
# Extracting title
sitcom_title = extract_title(sitcom_pitch)
print(sitcom_title)

Second Lock


In [14]:
scene_1_script = generate_scene_1_script(
    client=client,
    sitcom_title=sitcom_title,
    scene_description=scene_1_desc,
    rag_context=None,
    line_target=(55, 75)
)

In [15]:
display(Markdown(scene_1_script))


INT. JIMMY'S LOCKSMITH SHOP - DAY
[Opening theme fades out]

JIMMY, mid 40s, the quintessential old school locksmith with thick glasses and a magnifying glass on his head, is meticulously crafting a lock.

JIMMY
(whispering to himself)
Precision... Perfection...

Suddenly, the door swings open with a CREAK. In walks KAREN, late 30s, an overbearing but loveable customer.

KAREN
(excited)
Jimmy, I need a lock!

JIMMY
(startled)
Karen! You scared the life out of me!

[LAUGH TRACK]

KAREN
(smirks)
Just keeping you on your toes, Jimmy.

JIMMY
(sighs)
What can I do for you today, Karen?

KAREN
(flustered)
I need a lock for my diary. My husband keeps snooping!

JIMMY
(raised eyebrows)
Your diary? That's it?

KAREN
(defensive)
Yes, Jimmy. My thoughts are precious.

[LAUGH TRACK]

JIMMY
(mischievous smile)
Well, Karen, for precious thoughts, we need a precious lock.

Karen rolls her eyes. Jimmy gets back to work, showing off his skills as he creates a miniature lock.

JIMMY
(concentrated)
This is an art, Karen... a balance of strength and delicacy.

KAREN
(amused)
Strength and delicacy? Sounds like my mother's meatloaf.

[LAUGH TRACK]

Jimmy chuckles and hands over the finished lock to Karen.

JIMMY
(proud)
Here it is, Karen. The Fort Knox of diary locks.

KAREN
(surprised)
Oh, Jimmy! It's perfect!

She tries to give Jimmy a hug, but he steps back, protecting his workbench.

JIMMY
(slightly panicked)
Careful, Karen! I've got a delicate ecosystem here.

[LAUGH TRACK]

KAREN
(laughs)
Okay, okay! Just keep my diary safe, Jimmy.

She exits the shop, leaving Jimmy alone with his locks. He looks around, proudly.

JIMMY
(to himself)
Another day, another lock.

[LAUGH TRACK]

FADE OUT.

### 🗂️ Adding Metadata to the Vector Database

To extract structured information from each generated sitcom scene, we use th `summarize_scene` function. This function simulates the head writer of the show and analyzes the scene script to produce a concise summary along with key metadata.

- **Summary**: A 100–150 word overview of the scene’s key actions, character dynamics, and notable moments.
- **Characters**: A list of characters who appear or speak in the scene.
- **Location**: The main setting of the scene, if clearly identifiable (e.g., “locksmith shop,” “apartment hallway”).
- **Recurring Joke**: Any running gag or callback featured in the scene.
- **Emotional Tone**: A 1–2 word description of the scene’s mood (e.g., “chaotic,” “hopeful,” “awkward”).

The metadata is returned as a dictionary and appended to the `scene_metadata` list using the `add_scene_to_vector_db` function. This step is crucial: instead of supplying the full script to the next generation phase, we pass a concise summary and key elements to guide smoother, context-aware scene generation.


In [16]:
scene_1_summary = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_1_script
)

In [17]:
# Adds scene to the vector database
add_scene_to_vector_db(
    scene_1_summary,
    full_script=scene_1_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


In [18]:
print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


Total scenes stored in vector DB: 1 


Scene 1
Summary: In this scene, Jimmy, a dedicated locksmith, is crafting a lock when Karen, a regular customer, enters his shop. She startles him, and they exchange a few jokes, revealing a friendly, teasing relationship. Karen requests a lock for her diary to keep her husband from snooping, which Jimmy finds amusing but agrees to create. As he works, he explains his craft to Karen, comparing it to an art of strength and delicacy. Karen makes a joke, comparing it to her mother's meatloaf. Once the lock is ready, Karen tries to hug Jimmy, but he steps back, protecting his workbench. Karen leaves, and Jimmy, left alone, reflects on his day with satisfaction.
Characters: ['Jimmy', 'Karen']
Location: Jimmy's Locksmith Shop
Recurring Joke: None
Emotional Tone: Amusing


## 🎬 Scene 2 Generation

The next part is to generate Scene 2. This is one of the most important scenes in the script, as it’s where the ReAct (Reasoning + Acting) system comes into play.

Metadata from Scene 1 is retrieved and interpreted by the `CharacterAgent`, `ComedyAgent`, and `EnvironmentAgent`. Each agent reasons over the metadata independently and provides creative suggestions based on its area of expertise—such as character consistency, comedic dynamics, or environmental context.

These agent responses are then passed to the `ScenePlannerAgent`, which integrates their inputs and plans the next scene accordingly. This collaborative reasoning-and-acting process ensures that each scene builds logically and creatively on what came before.

The advantage of using ReAct-style agents over purely role-based agents is that each agent not only plays a role but also actively reasons and adapts based on prior context—making the generation more dynamic, context-aware, and narratively coherent.


In [19]:
scene_2_desc = extract_scene(outline, 2)
print(scene_2_desc)

Scene 2: "Unexpected Key" Zoe unexpectedly shows up at Jimmy's shop, surprising him and setting up their awkward reunion.


### 🎭 Character Agent

The first step is to analyze the characters using the `CharacterAgent`. This agent is responsible for tracking which characters appear in each scene, evaluating their consistency with past behavior, and recommending meaningful interactions to advance the story.

The `CharacterAgent` operates using a **ReAct workflow**, where each phase reflects a specific role in a sitcom writers’ room:

- **Think**: Identifies which characters are present in the current scene and determines the relevant prior scenes to draw context from — like a **Writers' Assistant** reviewing past episodes. This step also retrieves the relevant script metadata used to guide character evaluation.
- **Act**: Retrieves character histories by analyzing summaries from earlier scenes — like a **Script Coordinator** maintaining continuity.
- **Observe**: Checks whether characters behave consistently with their established traits and arcs — like a **Head Writer** reviewing for narrative logic.
- **Recommend**: Suggests specific character interactions that align with the scene’s tone or resolve inconsistencies — like a **Co-Executive Producer** steering character development.

This structure ensures character arcs are coherent, grounded, and creatively engaging throughout the scriptwriting process.


In [20]:
import importlib
import agents.character_agent
importlib.reload(agents.character_agent)
from agents.character_agent import CharacterAgent


In [21]:
'''
from typing import Tuple, Dict, List

def evaluate_character_agent_scene(
    agent: CharacterAgent,
    scene_description: str,
    scene_number: int
) -> Tuple[Dict[str, Dict], bool, str, str, List[str]]:
    """
    Runs the CharacterAgent on a given scene and prints a cleaned summary of results:
    - Consistency verdict
    - Short explanation
    - 2 recommended interactions
    - Agent's internal thoughts

    Args:
        agent (CharacterAgent): The CharacterAgent instance.
        scene_description (str): The current scene's description.
        scene_number (int): The scene number being evaluated.

    Returns:
        Tuple of:
            - character_histories (dict)
            - is_consistent (bool)
            - explanation (str)
            - recommendations (str)
            - internal_thoughts (list of str)
    """
    print(f"🔁 Agent will consider the last {agent.num_scenes} scene(s) for context.\n")

    character_histories, is_consistent, explanation, recommendations, thoughts = agent.run(
        scene_description=scene_description,
        scene_number=scene_number
    )

    # Extract short explanation only
    short_explanation = ""
    for line in explanation.split("\n"):
        if line.strip().startswith("2. Short Explanation Why:"):
            short_explanation = line.strip()
            break

    # Only keep the two interaction suggestions (strip label)
    clean_recommendations = "\n".join([
        line for line in recommendations.split("\n")
        if line.strip().startswith("1.") or line.strip().startswith("2.")
    ])

    print(f"Scene {scene_number} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}\n")
    print("Explanation:")
    print(short_explanation if short_explanation else explanation.strip())
    print("\nInteraction Improvement Recommendations:")
    print(clean_recommendations.strip())
    print("\nAgent's Internal Thoughts:")
    for thought in thoughts:
        print("-", thought)

    return character_histories, is_consistent, explanation, recommendations, thoughts

'''

'\nfrom typing import Tuple, Dict, List\n\ndef evaluate_character_agent_scene(\n    agent: CharacterAgent,\n    scene_description: str,\n    scene_number: int\n) -> Tuple[Dict[str, Dict], bool, str, str, List[str]]:\n    """\n    Runs the CharacterAgent on a given scene and prints a cleaned summary of results:\n    - Consistency verdict\n    - Short explanation\n    - 2 recommended interactions\n    - Agent\'s internal thoughts\n\n    Args:\n        agent (CharacterAgent): The CharacterAgent instance.\n        scene_description (str): The current scene\'s description.\n        scene_number (int): The scene number being evaluated.\n\n    Returns:\n        Tuple of:\n            - character_histories (dict)\n            - is_consistent (bool)\n            - explanation (str)\n            - recommendations (str)\n            - internal_thoughts (list of str)\n    """\n    print(f"🔁 Agent will consider the last {agent.num_scenes} scene(s) for context.\n")\n\n    character_histories, is_con

In [22]:
# Initialize the character agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=scene_metadata,
    num_scenes=1
)

# Run and store results from the character agent
char_histories, char_is_consistent, char_explanation, char_recommendations, char_thoughts = evaluate_character_agent_scene(
    agent=character_agent,
    scene_description=scene_2_desc,
    scene_number=2
)


🔁 Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Consistency: ✅ Consistent

Explanation:
2. Short Explanation Why: Based on the information provided, Jimmy's surprise at Zoe's unexpected arrival is consistent with his established personality traits. His dedication to his work might make him less prepared for personal surprises. Zoe's boldness in showing up unannounced is also consistent with her spontaneous and unpredictable nature. However, without more information about their past relationship or Zoe's speaking style, it's hard to fully assess the consistency.

Interaction Improvement Recommendations:
1. Zoe, in her spontaneous and bold style, could try to lighten the mood of the awkward reunion by cracking a joke about Jimmy's lock-making, similar to how he and Karen banter in scene 1. This interaction can show that Zoe is familiar with Jimmy's work and their shared past. — (This interaction is justified as we know from

### 🎤 Comedic Agent


The `ComedicAgent` is responsible for maintaining the tonal consistency of humor throughout the sitcom and making targeted punch-up recommendations to enhance comedic value.

This agent operates using a **ReAct workflow**, where each phase mimics a specific role in a sitcom writers’ room and draws on **prior scene metadata** for context:

- **Think**: Logs which scene is under review and determines which previous scenes will provide comedic context — like a **Writers' Assistant** flagging what material to reference.
- **Act**: Retrieves summaries and running jokes from recent scenes by referencing stored **vector metadata** — like a **Script Coordinator** tracking comedic beats.
- **Observe**: Evaluates whether the new scene's humor is consistent with previous tone and joke usage — like a **Head Writer** preserving the show's comedic style.
- **Recommend**: Suggests two realistic comedic improvements grounded in the scene’s context and character behavior — like a **Co-Executive Producer** overseeing punch-up work.

This design ensures each scene contributes to a cohesive comedic voice and builds on established humor organically, enhancing audience engagement.


**Maybe better note how this suggestions also based off inconsistency or not**

In [23]:
import importlib
import agents.comedy_agent
importlib.reload(agents.comedy_agent)
from agents.comedy_agent import ComedicAgent


In [25]:
# Initialize the comedy agent
comedic_agent = ComedicAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=1  # how many scenes to look back
)

# Run and store results from the comedy agent
com_context, com_is_consistent, com_analysis_text, com_recommendations, com_thoughts = evaluate_comedic_agent_scene(
    agent=comedic_agent,
    scene_description=scene_2_desc,
    scene_number=2
)

🔁 Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Comedic Tone: ❌ Inconsistent

Comedic Tone Analysis:
3. Short Explanation: The comedic tone in the new scene seems to be consistent with the previous scene. Both scenes involve Jimmy being surprised by a customer's arrival, leading to humorous interactions. The awkward humor in the current scene is a natural progression from the friendly, teasing humor in the previous scene.

Comedic Improvement Recommendations:
1. [Suggestion] - Zoe, upon entering the shop, should immediately start critiquing the layout and organization of Jimmy's shop in a teasing manner, suggesting to him that he could use her help. This could be a callback to a previous running joke about Jimmy's disorganization skills. The humor lies in Zoe's immediate critique, exaggerating Jimmy's lack of order. — (This suggestion builds on their existing dynamic of friendly teasing from previous scenes where Jimmy's 

### 🌆 Environment Agent

The `EnvironmentAgent` is responsible for ensuring spatial continuity and logical scene transitions in a sitcom setting. It verifies that each new scene's location aligns naturally with prior ones and suggests immersive environmental details to enhance tone and narrative flow.

This agent operates using a **ReAct workflow**, where each step mimics a professional role in a sitcom writers' room and draws on **prior scene metadata**:

- **Think**: Logs which scene is being reviewed and determines which previous environments will provide context — like a **Writers' Assistant** organizing location continuity.
- **Act**: Analyzes the current scene’s setting and identifies key environmental elements — like a **Script Coordinator** extracting staging needs from a scene.
- **Observe**: Evaluates whether the environment transition feels natural given recent scenes — like a **Head Writer** safeguarding spatial logic and pacing.
- **Recommend**: Suggests sensory or scenic details to support the environment or smooth jarring transitions — like a **Co-Executive Producer** fine-tuning atmosphere with practical details.

Together, these steps ensure that every scene change feels intentional, immersive, and narratively cohesive — supporting both visual consistency and comedic timing.



In [27]:
# TO DELETE
import importlib
import agents.environment_agent
importlib.reload(agents.environment_agent)
from agents.environment_agent import EnvironmentAgent


In [28]:
'''

from typing import Tuple, Dict, List

def evaluate_environment_agent_scene(
    agent,
    scene_description: str,
    scene_number: int
) -> Tuple[Dict, bool, str, str, List[str]]:
    """
    Runs the EnvironmentReActAgent on a given scene and prints a structured summary:
    - Whether the environment transition is logical
    - Explanation or critique
    - Suggested environment detail enhancements
    - Internal reasoning trace from the agent

    Args:
        agent: An initialized EnvironmentReActAgent
        scene_description (str): Description of the current scene
        scene_number (int): Scene number (for indexing and logging)

    Returns:
        Tuple containing:
            - context (dict): Location and analysis metadata
            - is_consistent (bool): Whether the transition is logical
            - explanation (str): The model's explanation
            - suggestions (str): Proposed environment enhancements
            - internal_thoughts (list): Agent's reasoning log
    """
    print(f"🔁 Environment Agent will consider the last {agent.num_scenes} scene(s) for context.\n")

    context, is_consistent, explanation, suggestions, thoughts = agent.run(
        scene_description=scene_description,
        scene_number=scene_number
    )

    print(f"Scene {scene_number} — Environment Transition: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}\n")

    print("Explanation:")
    print(explanation.strip())

    print("\nEnvironment Detail Suggestions:")
    print(suggestions.strip())

    print("\nEnvironment Agent's Internal Thoughts:")
    for thought in thoughts:
        print("-", thought)

    return context, is_consistent, explanation, suggestions, thoughts

'''

'\n\nfrom typing import Tuple, Dict, List\n\ndef evaluate_environment_agent_scene(\n    agent,\n    scene_description: str,\n    scene_number: int\n) -> Tuple[Dict, bool, str, str, List[str]]:\n    """\n    Runs the EnvironmentReActAgent on a given scene and prints a structured summary:\n    - Whether the environment transition is logical\n    - Explanation or critique\n    - Suggested environment detail enhancements\n    - Internal reasoning trace from the agent\n\n    Args:\n        agent: An initialized EnvironmentReActAgent\n        scene_description (str): Description of the current scene\n        scene_number (int): Scene number (for indexing and logging)\n\n    Returns:\n        Tuple containing:\n            - context (dict): Location and analysis metadata\n            - is_consistent (bool): Whether the transition is logical\n            - explanation (str): The model\'s explanation\n            - suggestions (str): Proposed environment enhancements\n            - internal_tho

In [32]:
# Initialize the environment agent
environment_agent = EnvironmentAgent(
    client=client,
    vector_metadata=vector_metadata,  # prior scene metadata
    num_scenes=1  # how many scenes to look back
)

context, is_consistent, explanation, env_recommendations, env_thoughts = evaluate_environment_agent_scene(
    agent=environment_agent,
    scene_description=scene_2_desc,
    scene_number=2
)

🔁 Environment Agent will consider the last 1 scene(s) for context.

📚 Retrieving script metadata for scene(s): [1]
Scene 2 — Environment Transition: ✅ Consistent

Explanation:
Short Explanation: The transition from "Jimmy's Locksmith Shop" to "Jimmy's Shop" is logical and believable. It's likely that "Jimmy's Shop" is just a shortened version of "Jimmy's Locksmith Shop", so the audience would understand that it's the same location. 
- Suggested Transition Setup: Not necessary in this case.

Environment Detail Suggestions:
Environment Details Suggestions:
- Suggestion 1: As the characters converse, the background noise of a key-making machine whirring and grinding intermittently could add a touch of realism to the scene. This could also create some funny moments, like a character getting startled or having to raise their voice over the noise. 
- Suggestion 2: A wall of countless keys hanging behind Jimmy could be visually interesting, and could lead to comedic moments where Jimmy can't 

### 🧠 Scene Planner Agent

The `ScenePlannerAgent` acts as the **Executive Producer**, responsible for synthesizing suggestions from all other agents and outlining a coherent, engaging plan for the next scene.

Rather than simply selecting from character, comedic, and environment recommendations, this agent **distills key themes and feedback** into a unified set of creative objectives that drive the sitcom forward.

The agent uses a **ReAct-based synthesis approach**, interpreting prior agent outputs and generating new, structured goals:

- **Character Goals (2)** — derived from the `CharacterAgent`'s recommendations, focusing on growth, conflict, or relationships.
- **Comedic Goal (1)** — distilled from tone analysis and punch-up suggestions made by the `ComedicAgent`.
- **Environment Detail (1)** — extracted from the `EnvironmentAgent`'s observations and setting enhancements.
- **Creative Suggestion (1)** — an original narrative nudge that naturally integrates all the above goals into a sitcom-appropriate next step.

This agent ensures every new scene is **purposeful, tonally consistent, and narratively aligned**, helping maintain momentum and coherence throughout the episode.


In [34]:
# TO DELETE
import importlib
import agents.scene_planner_agent
importlib.reload(agents.scene_planner_agent)
from agents.scene_planner_agent import ScenePlannerAgent

In [36]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_2_plan = scene_planner_agent.plan_next_scene(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_recommendations=env_recommendations,
    scene_number=2
)

In [37]:
display(Markdown(scene_2_plan))

Scene Plan:
Character Goals:
- [Goal 1] Showcase Zoe's bold and playful nature by having her break the tense atmosphere of the reunion with a joke about Jimmy's lock-making, subtly revealing her familiarity with his work and their shared history.
- [Goal 2] Have Jimmy regain composure after his initial surprise, revealing his professional and boundary-setting side by courteously inviting Zoe to share the reason for her visit.

Comedic Goal:
- [Goal] Inject humor into the scene by capitalizing on Jimmy's surprise at Zoe's visit, leading to physical comedy, followed by witty banter that references their shared past and ongoing jokes about Zoe's knack for dramatic entrances and Jimmy's disorganization.

Environment Detail:
- [Detail] Enhance the shop's atmosphere with the continuous sounds of a key-cutting machine in the background. This could be leveraged for comedic moments and to emphasize the reality of Jimmy's work environment.

Creative Suggestion:
- [Suggestion] As Zoe starts to explain her reason for visiting, the conversation is regularly interrupted by the loud noise of the key-cutting machine, leading to funny exchanges and misunderstandings. Meanwhile, Jimmy tries to find a specific key from his disorganized wall of keys, creating physical comedy and an opportunity for Zoe to offer her assistance in a teasing manner.

In [38]:
print(scene_2_plan)

Scene Plan:
Character Goals:
- [Goal 1] Showcase Zoe's bold and playful nature by having her break the tense atmosphere of the reunion with a joke about Jimmy's lock-making, subtly revealing her familiarity with his work and their shared history.
- [Goal 2] Have Jimmy regain composure after his initial surprise, revealing his professional and boundary-setting side by courteously inviting Zoe to share the reason for her visit.

Comedic Goal:
- [Goal] Inject humor into the scene by capitalizing on Jimmy's surprise at Zoe's visit, leading to physical comedy, followed by witty banter that references their shared past and ongoing jokes about Zoe's knack for dramatic entrances and Jimmy's disorganization.

Environment Detail:
- [Detail] Enhance the shop's atmosphere with the continuous sounds of a key-cutting machine in the background. This could be leveraged for comedic moments and to emphasize the reality of Jimmy's work environment.

Creative Suggestion:
- [Suggestion] As Zoe starts to ex

### Generating Scene 2 Script

In [50]:
import importlib
import screen_writing
importlib.reload(screen_writing)


<module 'screen_writing' from '/content/drive/MyDrive/Spring 2025/Gen AI with LLM/Project/utils/screen_writing.py'>

In [51]:
scene_2_script = generate_scene(
    client=client,
    scene_plan=scene_2_plan,
    scene_number= 2
)

print(scene_2_script)

INT. JIMMY'S LOCKSMITH SHOP - DAY

Jimmy, a middle-aged locksmith, is hunched over his workbench, engrossed in his work. The sound of a KEY-CUTTING MACHINE fills the room. Suddenly the door swings open with a bang.

Zoe, an expressive woman in her late thirties, stands in the doorway, striking a dramatic pose.

ZOE: (grinning) "Remember when you said you'd make me a key to your heart, Jimmy?"

Jimmy jumps in surprise, knocking a pile of keys off the workbench, creating a clatter.

JIMMY: (flustered) "Zoe! You scared the life out of me."

Zoe chuckles, stepping over the scattered keys to approach Jimmy.

ZOE: (teasing) "Well, someone's got to keep you on your toes, Jimmy. Besides, you've always been a mess."

Jimmy regains his composure, standing tall as he meets Zoe's playful gaze.

JIMMY: (smiling) "I see you haven't lost your dramatic flair, Zoe. What brings you here?"

ZOE: (grinning) "Well, I was in town and thought I'd visit my favorite locksmith."

Just as Zoe starts to speak aga

In [52]:
display(Markdown(scene_2_script))

INT. JIMMY'S LOCKSMITH SHOP - DAY

Jimmy, a middle-aged locksmith, is hunched over his workbench, engrossed in his work. The sound of a KEY-CUTTING MACHINE fills the room. Suddenly the door swings open with a bang.

Zoe, an expressive woman in her late thirties, stands in the doorway, striking a dramatic pose.

ZOE: (grinning) "Remember when you said you'd make me a key to your heart, Jimmy?"

Jimmy jumps in surprise, knocking a pile of keys off the workbench, creating a clatter.

JIMMY: (flustered) "Zoe! You scared the life out of me."

Zoe chuckles, stepping over the scattered keys to approach Jimmy.

ZOE: (teasing) "Well, someone's got to keep you on your toes, Jimmy. Besides, you've always been a mess."

Jimmy regains his composure, standing tall as he meets Zoe's playful gaze.

JIMMY: (smiling) "I see you haven't lost your dramatic flair, Zoe. What brings you here?"

ZOE: (grinning) "Well, I was in town and thought I'd visit my favorite locksmith."

Just as Zoe starts to speak again, the KEY-CUTTING MACHINE cuts her off with a loud WHIRRING sound. They both wince, then laugh.

JIMMY: (over the noise) "I didn't catch that, Zoe!"

Zoe gestures for Jimmy to turn off the machine. Jimmy, in his confusion, accidentally sends another pile of keys flying.

ZOE: (laughing) "Still the king of chaos I see!"

Jimmy finally turns off the machine, shaking his head with a grin.

JIMMY: (chuckles) "Alright, alright. What was that you were saying?"

Zoe glances at the scattered keys and then at Jimmy's disorganized wall of keys.

ZOE: (grinning) "I said, I need a key made. But by the looks of it, you might need my help finding the right one!"

They both burst out laughing, the tension from their reunion completely dissolved, replaced by the familiar rhythm of their shared past.

### Adding Scene 2 to Vector Database

In [41]:
scene_metadata_2 = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_2_script
)

# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata_2,
    full_script=scene_2_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)

print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


Total scenes stored in vector DB: 2 


Scene 1
Summary: In this scene, Jimmy, a dedicated locksmith, is crafting a lock when Karen, a regular customer, enters his shop. She startles him, and they exchange a few jokes, revealing a friendly, teasing relationship. Karen requests a lock for her diary to keep her husband from snooping, which Jimmy finds amusing but agrees to create. As he works, he explains his craft to Karen, comparing it to an art of strength and delicacy. Karen makes a joke, comparing it to her mother's meatloaf. Once the lock is ready, Karen tries to hug Jimmy, but he steps back, protecting his workbench. Karen leaves, and Jimmy, left alone, reflects on his day with satisfaction.
Characters: ['Jimmy', 'Karen']
Location: Jimmy's Locksmith Shop
Recurring Joke: None
Emotional Tone: Amusing

Scene 2
Summary: In the locksmith shop, Jimmy is cutting a key when Zoe enters, surprising him. Their playful banter reveals a shared history and familiarity. Zoe teases Jimmy about his

## 🎬 Scene 3 Generation

Scene 3 (through 20) will follow the same exact structure as Scene 2

In [53]:
scene_3_desc = extract_scene(outline, 3)
print(scene_3_desc)

Scene 3: "The First Challenge" Zoe challenges Jimmy's traditional thinking by suggesting improvements to his locksmith shop, creating a tension-filled funny moment.


### Character Agent

### Comedy Agent

### Enviroment Agent


### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_3_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=3
)

In [ ]:
print(scene_3_plan)

### Generating Scene 3 Script

In [ ]:
scene_3_script = generate_scene(
    client=client,
    scene_plan=scene_3_plan,
    scene_number= 3
)

print(scene_3_script)

### Adding Metadata to Vector Database

In [ ]:
'''
MAKE FUNCTION!!!
'''

scene_metadata_3 = summarize_scene(
    client=client,
    sitcom_title=sitcom_title,
    scene_script=scene_3_script
)


# Adds scene to the vector database
add_scene_to_vector_db(
    scene_metadata_3,
    full_script=scene_3_script,
    embedding_model=embedding_model,
    index=index,
    vector_metadata=vector_metadata
)


print("Total scenes stored in vector DB:", index.ntotal, "\n")

for i, meta in enumerate(vector_metadata):
    print(f"\nScene {i + 1}")
    print("Summary:", meta["summary"])
    print("Characters:", meta["characters"])
    print("Location:", meta["location"])
    print("Recurring Joke:", meta["recurring_joke"])
    print("Emotional Tone:", meta["emotional_tone"])


## Scene 4 Generation

In [ ]:
scene_4_desc = extract_scene(outline, 4)
print(scene_4_desc)

### Character Agent

In [ ]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_4_desc,
    scene_number=4
)

# Outputs
print(f"Scene {4} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


### Comedy Agent

In [ ]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_4_desc,
    scene_number=4
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

### Enviroment Agent


In [ ]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=3
)

In [ ]:
environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_4_desc,
    scene_number=4
)

In [ ]:
print(environment_details_suggestions)

### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_4_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=4
)

In [ ]:
print(scene_4_plan)

### Generating Scene 4 Script

In [ ]:
scene_4_script = generate_scene(
    client=client,
    scene_plan=scene_4_plan,
    scene_number= 4
)

print(scene_4_script)

## Scene 5 Generation

In [ ]:
scene_5_desc = extract_scene(outline, 5)
print(scene_5_desc)

### Character Agent

In [ ]:
# Initialize the agent
character_agent = CharacterAgent(
    client=client,
    vector_metadata=vector_metadata,
    num_scenes=3
)

character_histories, is_consistent, explanation, char_recommendations, thoughts = character_agent.run(
    scene_description=scene_5_desc,
    scene_number=5
)

# Outputs
print(f"Scene {5} — Consistency: {'✅ Consistent' if is_consistent else '❌ Inconsistent'}")
print("\nExplanation:\n", explanation)

print("\nInteraction Improvement Recommendations:\n", char_recommendations)

print("\nAgent's Internal Thoughts:")
for thought in thoughts:
    print("-", thought)


### Comedy Agent

In [ ]:
comedic_agent = ComedicAgent(client=client, vector_metadata=vector_metadata)

is_consistent, analysis_text, com_recommendations, thoughts = comedic_agent.run(
    scene_description=scene_5_desc,
    scene_number=5
)

print("Is Consistent:", is_consistent)
print("Analysis:\n", analysis_text)
print("Recommendations:\n", com_recommendations)
print("Internal Thoughts:\n", thoughts)

### Enviroment Agent


In [ ]:
environment_agent = EnvironmentReActAgent(
    client=client,
    vector_metadata=vector_metadata,  # your list of prior scene metadata
    num_scenes=3
)

environment_analysis, transition_check, environment_details_suggestions, env_thoughts = environment_agent.run(
    scene_description=scene_5_desc,
    scene_number=5
)

print(environment_details_suggestions)

### Scene Planner Agent

In [ ]:
scene_planner_agent = ScenePlannerAgent(client=client)

scene_5_plan = scene_planner_agent.plan_next_scene_explicit(
    character_recommendations=char_recommendations,
    comedic_recommendations=com_recommendations,
    environment_details_suggestions=environment_details_suggestions,
    scene_number=5
)

print(scene_5_plan)

### Generating Scene 5 Script

In [ ]:
scene_5_script = generate_scene(
    client=client,
    scene_plan=scene_5_plan,
    scene_number= 5
)

print(scene_5_script)

## Combining Scripts



In [ ]:
# List your scene scripts in order
scene_scripts = [
    scene_1_script,
    scene_2_script,
    scene_3_script,
    scene_4_script,
    scene_5_script
]

# Combine with optional scene headers
full_episode_script = "\n\n".join([
    f"### Scene {i+1} ###\n{script.strip()}"
    for i, script in enumerate(scene_scripts)
])

print(full_episode_script)